In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/flyhopper/scripts/Julia/Project.toml`


In [2]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays

WebIO._IJuliaInit()

In [3]:
l0 = 0.15
l1 = 0.3

l2 = 0.3
l3 = 0.15
l4 = 0.15
l34 = l3+l4

m0 = 1.0
m1 = 1.0
m2 = 1.0
m3 = 1.0

g = 9.81

# I1 = m1*(l1^2)
# I2 = m2*(l2^2)

9.81

In [4]:
world = RigidBody{Float64}("world")
doublependulum = Mechanism(world; gravity = SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

In [5]:
curdir = pwd()
urdfpath = joinpath(curdir, "../../res/flyhopper_parallel/urdf/flyhopper_parallel.urdf")
doublependulum = parse_urdf(urdfpath, floating=false)
state = MechanismState(doublependulum)

MechanismState{Float64, Float64, Float64, …}(…)

In [103]:
function f_1(x)
    # double-double pendulum dynamics
    q0 = x[1]
    q0d = x[2]
    q1 = x[3]
    q1d = x[4]
    q2 = x[5]
    q2d = x[6]
    q3 = x[7]
    q3d = x[8]
        
    # Mass matrix
    M = [ 1.0*l0^2*m0 + l0^2*m1 + 2*l0*l1*m1*cos(q1) + l1^2*m1  l0*l1*m1*cos(q1) + l1^2*m1  0  0;
          l0*l1*m1*cos(q1) + l1^2*m1  l1^2*m1  0  0;
          0  0  1.0*l2^2*m2 + l2^2*m3 + 2*l2*l34*m3*cos(q3) + l34^2*m3  l2*l34*m3*cos(q3) + l34^2*m3;
          0  0  l2*l34*m3*cos(q3) + l34^2*m3  l34^2*m3]
    
    # Coriolis term
    C = [ -2*l0*l1*m1*q0d*q1d*sin(q1) - l0*l1*m1*q1d^2*sin(q1);
          l0*l1*m1*q0d^2*sin(q1);
          -2*l2*l34*m3*q2d*q3d*sin(q3) - l2*l34*m3*q3d^2*sin(q3);
          l2*l34*m3*q2d^2*sin(q3)]
    
    # Gravity term
    G = [g*l0*m0*cos(q0) + g*l0*m1*cos(q0) + g*l1*m1*cos(q0 + q1);
         g*l1*m1*cos(q0 + q1);
         g*l2*m2*cos(q2) + g*l2*m3*cos(q2) + g*l34*m3*cos(q2 + q3);
         g*l34*m3*cos(q2 + q3)]
    #=
    qd = zeros(4)
    qd[1] = q0d
    qd[2] = q1d
    qd[3] = q2d
    qd[4] = q3d
    =#
    qdd = M\(- G - C)
    q0dd = qdd[1]
    q1dd = qdd[2]
    q2dd = qdd[3]
    q3dd = qdd[4]
    
    ẋ = zeros(8)
    ẋ[1] = q0d # q0 dot
    ẋ[2] = q0dd # q0 double dot
    ẋ[3] = q1d # q1 dot
    ẋ[4] = q1dd # q1 double dot
    ẋ[5] = q2d # q2 dot
    ẋ[6] = q2dd # q2 double dot
    ẋ[7] = q3d # q3 dot
    ẋ[8] = q3dd # q3 double dot
    
    return ẋ
end

f_1 (generic function with 1 method)

In [104]:
function rk4_step(f,xk,h)
    a = [0 0 0 0; 
         0.5 0 0 0; 
         0 0.5 0 0; 
         0 0 1 0]
    b = [1/6, 1/3, 1/3, 1/6]
    f1 = f(xk)
    f2 = f(xk .+ h*a[2, 1]*f1)
    f3 = f(xk .+ h*a[3, 1]*f1 .+ h*a[3, 2]*f2)
    f4 = f(xk .+ h*a[4, 1]*f1 .+ h*a[4, 2]*f2 .+ h*a[4, 3]*f3)
    xn = xk .+ h*(b[1]*f1 .+ b[2]*f2 .+ b[3]*f3 .+ b[4]*f4)
    
    return xn
end

rk4_step (generic function with 1 method)

In [105]:
Tf = 10.0
h = 0.001 #20 Hz
N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1));

In [106]:
function simulate!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= rk4_step(f_1, xtraj[:,k], h)
    end
end

simulate! (generic function with 1 method)

In [107]:
x0 = [-30*pi/180; 0.0; -120*(pi/180); 0.0; -150*(pi/180); 0.0; 120*(pi/180); 0.0]
xtraj = zeros(8,N)
xtraj[:,1] = x0;

In [108]:
simulate!(xtraj, N)

In [109]:
q0 = -xtraj[1, :] .- 30*(pi/180)
q1 = -xtraj[3, :] .- 120*(pi/180)
q2 = -xtraj[5, :] .- 150*(pi/180)
q3 = -xtraj[7, :] .+ 120*(pi/180)

# qs = [SVector{4}(x) for x in eachrow([q0 q1 q2 q3])]
qs = convert(AbstractArray{Float64}, [q2 q0 q3 q1]) 
# qs = convert(AbstractArray{Float64, 4}, [q0 q1 q2 q3]) 
ts = convert(AbstractVector{Float64}, thist) # AbstractVector(thist)
q_array = [ qs[i,:] for i in 1:size(qs,1)] 

10001-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0]
 [-1.2136713720689585e-5, 2.427336939681002e-5, 2.02279391450233e-5, -3.2364673114848586e-5]
 [-4.854669022247293e-5, 9.709245181299675e-5, 8.091247744568264e-5, -0.0001294594917036207]
 [-0.00010922943568836274, 0.00021845417083954377, 0.00018205577760976865, -0.00029128685411938804]
 [-0.0001941841276393852, 0.00038835340216303305, 0.00032366144452167234, -0.0005178507596799165]
 [-0.00030340961575792136, 0.0006067829781186695, 0.000505734525794832, -0.0008091568117647441]
 [-0.00043690442304100685, 0.0008737336940642937, 0.0007282815125506659, -0.0011652122221499361]
 [-0.0005946667472835898, 0.0011891943165752705, 0.000991310340419549, -0.0015860258165858454]
 [-0.0007766944628899708, 0.0015531515934606999, 0.0012948303907638348, -0.0020716080416196547]
 [-0.0009829851230174214, 0.001965590265600503, 0.001638852492128251, -0.0026219709726662543]
 [-0.0012135359620484287, 0.002426493080603498, 0.0020233889219136714, -0.003

In [13]:
# mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true, inertias=false));
mvis = MechanismVisualizer(doublependulum, URDFVisuals(urdfpath));

render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x20, 0x31, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), "Link 3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 4" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72

In [95]:
# set_configuration!(mvis, [x0[1]-150*(pi/180), x0[3]+120*(pi/180)])
# [q2 q0 q3 q1
set_configuration!(mvis, [-x0[5]-150*(pi/180), -x0[1]-30*(pi/180), -x0[7]+120*(pi/180), -x0[3]-120*(pi/180)])
# set_configuration!(mvis, [0, 0])

In [112]:
# Now we can simply call `simulate`, which will return a tuple consisting of:
# * simulation times (a `Vector` of numbers)
# * joint configuration vectors (a `Vector` of `Vector`s)
# * joint velocity vectors (a `Vector` of `Vector`s)

# MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);
animation = Animation(mvis, ts, -q_array)
setanimation!(mvis, animation)